## RANDOM TESTS

In [1]:
import numpy as np
import pandas as pd
import rdflib as rdf
import requests
import json
import re
import sys
import SPARQLWrapper as sw
from AnilistPython import Anilist

In [41]:
WIKIDATA_URL = "https://query.wikidata.org/sparql"

In [42]:
def get_query_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = sw.SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(sw.JSON)
    return sparql.query().convert()

In [50]:
anime_name_query = "attack"

# get the anime id from wikidata
query = """
SELECT ?anime ?animeLabel ?anilist_id ?default_review_score WHERE {
  ?anime wdt:P31/wdt:P279* wd:Q1107 .
  ?anime wdt:P8729 ?anilist_id .
  ?anime rdfs:label ?animeLabel.

  # stuff that may or not be there
  OPTIONAL{?anime wdt:P444 ?default_review_score}
  
  # filters
  FILTER(LANG(?animeLabel) = "en")
  FILTER(REGEX(?animeLabel, "%s", "i"))
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" . }
} ORDER BY ?animeLabel
""" % anime_name_query

results = get_query_results(WIKIDATA_URL, query)

print(json.dumps(results, indent=2, sort_keys=True))


{
  "head": {
    "vars": [
      "anime",
      "animeLabel",
      "anilist_id",
      "default_review_score"
    ]
  },
  "results": {
    "bindings": [
      {
        "anilist_id": {
          "type": "literal",
          "value": "1550"
        },
        "anime": {
          "type": "uri",
          "value": "http://www.wikidata.org/entity/Q98915343"
        },
        "animeLabel": {
          "type": "literal",
          "value": "Attack No. 1",
          "xml:lang": "en"
        },
        "default_review_score": {
          "type": "literal",
          "value": "6.9/10"
        }
      },
      {
        "anilist_id": {
          "type": "literal",
          "value": "16498"
        },
        "anime": {
          "type": "uri",
          "value": "http://www.wikidata.org/entity/Q22126305"
        },
        "animeLabel": {
          "type": "literal",
          "value": "Attack on Titan",
          "xml:lang": "en"
        }
      },
      {
        "anilist_id": {
        

In [51]:
# get the entities found in the query
entities = []
for result in results["results"]["bindings"]:
    entities.append(result["anime"]["value"].split("/")[-1])
print(entities)

['Q98915343', 'Q22126305', 'Q108860715', 'Q30931897', 'Q115731296', 'Q62076812', 'Q1363770']


In [52]:
# for each entity in entities, get the genres, anilist id
anilist_ids = []
for entity in entities:
    query = """
    SELECT ?anilist_id WHERE {
        ?anime wdt:P31/wdt:P279* wd:Q1107 .
        ?anime wdt:P8729 ?anilist_id .
        FILTER(?anime = wd:%s)
    }
    """ % entity
    results = get_query_results(WIKIDATA_URL, query)
    anilist_ids.append(results["results"]["bindings"][0]["anilist_id"]["value"])

print(anilist_ids)


['1550', '16498', '21281', '104723', '140596', '636', '87']


In [55]:
# for each anilist id, get the genres and the average score
genres = []
scores = []
anilist = Anilist()
for anilist_id in anilist_ids:
    anime = anilist.get_anime_with_id(anilist_id)
    genres.append(anime["genres"])
    scores.append(anime["average_score"])

print(genres)
print(scores)

[['Drama', 'Sports'], ['Action', 'Drama', 'Fantasy', 'Mystery'], ['Comedy', 'Fantasy', 'Slice of Life'], ['Action', 'Adventure', 'Comedy', 'Ecchi', 'Fantasy'], ['Comedy', 'Romance', 'Slice of Life'], ['Adventure', 'Comedy', 'Drama'], ['Drama', 'Mecha', 'Sci-Fi']]
[61, 84, 69, 52, 74, 62, 74]


In [56]:
# get the average score per genre
genre_scores = {}
for i in range(len(genres)):
    for genre in genres[i]:
        if genre not in genre_scores:
            genre_scores[genre] = []
        genre_scores[genre].append(scores[i])
print(genre_scores)

{'Drama': [61, 84, 62, 74], 'Sports': [61], 'Action': [84, 52], 'Fantasy': [84, 69, 52], 'Mystery': [84], 'Comedy': [69, 52, 74, 62], 'Slice of Life': [69, 74], 'Adventure': [52, 62], 'Ecchi': [52], 'Romance': [74], 'Mecha': [74], 'Sci-Fi': [74]}


In [5]:
# get imdb score for Neon Genesis Evangelion
# entity Q662
# imdb id tt0112159
anilist = Anilist()
anime = anilist.get_anime_with_id(int("16498"))
anime_score = anilist.get_anime_with_id(int("16498"))["average_score"]/100
anime_genres = anilist.get_anime_with_id(int("16498"))["genres"]
anime_image = anilist.get_anime_with_id(int("16498"))['cover_image']
print(anime.keys())
print(anime_score)
print(anime_image)


dict_keys(['name_romaji', 'name_english', 'starting_time', 'ending_time', 'cover_image', 'banner_image', 'airing_format', 'airing_status', 'airing_episodes', 'season', 'desc', 'average_score', 'genres', 'next_airing_ep'])
0.84
https://s4.anilist.co/file/anilistcdn/media/anime/cover/medium/bx16498-C6FPmWm59CyP.jpg
